In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
train = pd.read_csv('ybigta_sdss_train.csv',index_col='id')
train = train.reset_index()

In [7]:
# RobustScaler
from sklearn.preprocessing import RobustScaler
train_for_scaling = train.iloc[:,2:] #type과 fiberID 제외하고 데이터 생성
robustScaler = RobustScaler()
print(robustScaler.fit(train_for_scaling))
train_robustscaled = robustScaler.transform(train_for_scaling) #robustscaling 적용
train_robustscaled = pd.DataFrame(train_robustscaled,columns=train_for_scaling.columns)
train_robustscaled = pd.concat([train[['type', 'fiberID']], train_robustscaled], axis=1) 
train_robustscaled #robustscaling과 type,fiberID 합치기

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)


,type,fiberID,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,QSO,106,-0.666667,-0.435893,-0.291574,-0.061425,-0.111254,0.045898,-0.526436,-0.469058,...,-0.878988,-0.808048,-0.702073,-0.717412,-0.654057,-1.076404,-0.950766,-0.802411,-0.819421,-0.727297
1,QSO,492,0.513761,-0.580244,-0.413752,-0.220812,-0.112515,0.093583,-0.599909,-0.368806,...,-0.453638,-0.124812,0.072741,0.148958,0.295791,-0.463942,-0.125875,0.075270,0.155184,0.285476
2,QSO,388,0.195719,-0.185463,-0.377135,-0.229722,-0.127228,0.040244,-0.152645,-0.314845,...,0.026496,-0.095064,0.083110,0.147472,0.251690,0.011104,-0.094863,0.073686,0.147607,0.248041
3,QSO,531,0.633028,0.616269,0.437863,0.607836,0.845503,0.851924,0.742433,0.546231,...,0.794639,0.615596,0.795158,0.924695,0.971942,0.968832,0.616370,0.785311,0.919010,0.899853
4,QSO,180,-0.440367,0.928778,0.372564,0.167249,0.202501,0.223579,1.037113,0.462229,...,1.153932,0.575626,0.408968,0.413842,0.363370,1.370472,0.562681,0.404408,0.398004,0.386068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171960,STAR_PN,433,0.333333,-0.162202,0.146883,0.167690,0.689451,0.501561,-0.215678,0.119500,...,-0.065436,0.224207,0.267896,0.607375,0.488257,-0.086797,0.233696,0.267710,0.638486,0.484584
171961,STAR_PN,222,-0.311927,-0.287677,-0.160569,0.209511,0.968814,2.442132,-0.351929,-0.209937,...,-0.311846,-0.498241,-0.040090,0.610949,2.175321,-0.220553,-0.028332,0.310742,0.872800,2.161041
171962,STAR_PN,364,0.122324,0.310819,0.326378,0.394435,0.573275,0.648009,0.404805,0.403580,...,0.724278,0.539605,0.612140,0.700288,0.765620,0.619425,0.523664,0.601724,0.702361,0.728548
171963,STAR_PN,439,0.351682,-0.162202,0.146883,0.167690,0.689451,0.501561,-0.215678,0.119500,...,-0.065436,0.224207,0.267896,0.607375,0.488257,-0.086797,0.233696,0.267710,0.638486,0.484584


In [8]:
#X,y 분류
from sklearn.model_selection import train_test_split
X = train_robustscaled.drop(['type'], axis = 1)
y = train_robustscaled['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [9]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(objective='binary')

param_dist = {'boosting_type':['rf'],
              'n_estimators': [1500,2000,2500],
              'max_depth': [15,30,45],
              'learning_rate': [0.01,0.05,0.1],
              'bagging_freq': [1],
              'bagging_fraction':[0.9]
             }
grid_search = GridSearchCV(lgbm_model, n_jobs=6, param_grid=param_dist, cv=5, scoring="neg_log_loss", verbose=5)
grid_search.fit(X,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed: 27.0min
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed: 282.1min
[Parallel(n_jobs=6)]: Done 135 out of 135 | elapsed: 623.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31,
                                      objective='binary', random_state=None,
                                      reg_alpha=0.0, reg_lambda=0.0,
                                      silent=True, subsample=1.0,
                                      subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=6,
             param_grid={'bagging_fraction': [0.9], 'ba

In [14]:
grid_search.best_estimator_

LGBMClassifier(bagging_fraction=0.9, bagging_freq=1, boosting_type='rf',
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               learning_rate=0.01, max_depth=15, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=1500,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)